In [1]:
# Import libraries

import pandas as pd
import altair as alt

In [2]:
# Import datasets

# CPI: consumer price index (https://fred.stlouisfed.org/series/CPIAUCSL)
cpi_data = pd.read_csv('./data/CPIAUCSL.csv')

# PCE: personal consumption expenditures (https://fred.stlouisfed.org/series/PCE)
pce_data = pd.read_csv('./data/PCE.csv')

# SAVINGS: personal saving rate (https://fred.stlouisfed.org/series/PSAVERT)
savings_data = pd.read_csv('./data/PSAVERT.csv')

# REV CREDIT: revolving consumer credit (https://fred.stlouisfed.org/series/REVOLSL)
credit_data = pd.read_csv('./data/REVOLSL.csv')

# UNEMPL: unemployment rate (https://fred.stlouisfed.org/series/UNRATE)
unemployment_data = pd.read_csv('./data/UNRATE.csv')

# FED FUND RATE: interest rate (https://www.federalreserve.gov/monetarypolicy/openmarket.htm)
interest_data = pd.read_csv('./data/DFEDTARU.csv')

In [3]:
interest_data.head(2)

,DATE,DFEDTARU
0,2017-10-09,1.25
1,2017-10-10,1.25


In [4]:
def normalize_col(df, col_name):
    mean = df.mean().loc[col_name]  
    std = df.std().loc[col_name]
    df[col_name] = df[col_name].apply(lambda x: (x - mean) / std)

    return df

def get_combined_df(df_list, months):
    combined_df = None
    
    for df in df_list:
        start = df['DATE'].size - months
        df = df[start:]
        #df = normalize_col(df, df.columns[1])
        print(df.tail(2))
        
        combined_df = df if combined_df is None else combined_df.merge(df, on='DATE', how='outer')
        
    return combined_df

def get_combined_df_norm(df_list_norm, months):
    combined_df_norm = None
    
    for df in df_list_norm:
        start = df['DATE'].size - months
        df = df[start:]
        df = normalize_col(df, df.columns[1])
        print(df.tail(2))
        
        combined_df_norm = df if combined_df_norm is None else combined_df_norm.merge(df, on='DATE', how='outer')
        
    return combined_df_norm

In [ ]:
df_combined = get_combined_df_norm([cpi_data, pce_data, savings_data, credit_data, unemployment_data], 60)
df_combined = df_combined.rename(columns={'CPIAUCSL':'CPI', 'PSAVERT':'Savings', 'REVOLSL':'Revolving Credit', 'UNRATE':'Unemployment'})
df_combined = df_combined.melt(id_vars=['DATE'],var_name='INDEX')

In [ ]:
def get_indicator_chart():
    line = alt.Chart(df_combined).mark_line().encode(
        x='DATE',
        y='value',
        color='INDEX',
    )

    line_interest = alt.Chart(interest_data[2:]).mark_line(color='#000000').encode(
        x='DATE',
        y=alt.Y('INTEREST', title='value'),
    )

    point = alt.Chart(interest_data[2:]).mark_point(size=50).encode(
        x='DATE',
        y=alt.Y('INTEREST', title='value'),
    )

    chart = (line + line_interest + point).properties(
        width=800,
        title='Economic Indicators (Normalized, Except Interest)'
    )
    return chart

In [ ]:
indicator_chart = get_indicator_chart()
indicator_chart

### Consumer Price Index & Personal Consumption Expenditure

In [ ]:
df_CPI_PCE = get_combined_df_norm([cpi_data, pce_data], 60)
df_CPI_PCE = df_CPI_PCE.rename(columns={'CPIAUCSL':'CPI'})
df_CPI_PCE = df_CPI_PCE.melt(id_vars=['DATE'],var_name='INDEX')

df_CPI_PCE['DATE'] = pd.to_datetime(df_CPI_PCE['DATE'])
df_CPI_PCE = df_CPI_PCE[df_CPI_PCE['DATE'] >= '2020-08-01']

df_CPI_PCE.head()
df_CPI_PCE.dtypes

In [ ]:
# CPI vs PCE dataframe
df_CPI_PCE = get_combined_df_norm([cpi_data, pce_data], 60)
df_CPI_PCE = df_CPI_PCE.rename(columns={'CPIAUCSL':'CPI'})
df_CPI_PCE['DATE'] = pd.to_datetime(df_CPI_PCE['DATE'])
df_CPI_PCE = df_CPI_PCE[df_CPI_PCE['DATE'] >= '2020-08-01']

# CPI vs PCE Correlation
corr_matrix_CPI_PCE = df_CPI_PCE.corr()
corr_CPI_PCE = corr_matrix_CPI_PCE.iloc[0][1]

# CPI vs PCE dataframe melted
df_CPI_PCE = df_CPI_PCE.melt(id_vars=['DATE'],var_name='INDEX')

In [ ]:
# CPI vs PCE Graph

cpi_pce_line = alt.Chart(df_CPI_PCE).mark_line().encode(
    x=alt.X('DATE:T', title=None),
    y=alt.Y('value:Q', title=None),
    color=alt.Color('INDEX', title='') #legend=alt.Legend(legendX=10,legendY=2, 
)

cpi_pce_corr_text = alt.Chart({'values':[{}]}).mark_text(
    align='left', baseline='bottom'
).encode(
    x=alt.value(35), 
    y=alt.value(60), 
    text=alt.value([f"r: {corr_CPI_PCE:.3f}"]))

(cpi_pce_line+cpi_pce_corr_text).configure_legend(
    orient='top-left'
).properties(
    width=450, 
    height=225,
    title={'text':'Consumer Price Index vs Personal Consumption Expenditure',
           'subtitle':'A 2-Year Metric Comparison'})

### Personal Savings Rate & Revolving Credit

In [19]:
# Revolving Credt & Savings dataframe
df_REV_SAV = get_combined_df([savings_data, credit_data], 60)
df_REV_SAV = df_REV_SAV.rename(columns={'PSAVERT':'Savings', 'REVOLSL':'Revolving Credit'})
df_REV_SAV['DATE'] = pd.to_datetime(df_REV_SAV['DATE'])
df_REV_SAV = df_REV_SAV[df_REV_SAV['DATE'] >= '2020-08-01']

# Dataframe melted
df_REV_SAV = df_REV_SAV.melt(id_vars=['DATE'],var_name='INDEX')
df_REV_SAV.head()

           DATE  PSAVERT
761  2022-06-01      5.0
762  2022-07-01      5.0
           DATE     REVOLSL
653  2022-06-01  1125.65933
654  2022-07-01  1136.56209


,DATE,INDEX,value
0,2020-08-01,Savings,15.0
1,2020-09-01,Savings,14.3
2,2020-10-01,Savings,13.6
3,2020-11-01,Savings,13.0
4,2020-12-01,Savings,14.0


### Inflation

Causes of Inflation: https://news.stanford.edu/2022/09/06/what-causes-inflation/

In [45]:
# inflation dataframe
df_INFL = get_combined_df([cpi_data], 60)
df_INFL = df_INFL.rename(columns={'CPIAUCSL':'CPI'})
df_INFL['DATE'] = pd.to_datetime(df_INFL['DATE'])
df_INFL = df_INFL[df_INFL['DATE'] >= '2018-08-01']

# Calculate 1 month and 12 month lags, for later inflation % calculations
df_INFL['lag_1_diff'] = df_INFL['CPI'].diff()
df_INFL['lag_12_diff'] = df_INFL['CPI'].diff(periods=12)
df_INFL['lag_1'] = df_INFL['CPI'] - df_INFL['lag_1_diff']
df_INFL['lag_12'] = df_INFL['CPI'] - df_INFL['lag_12_diff']

# Calculate inflation as % increase MoM and YoY
df_INFL['MoM_inflation_perc'] = (df_INFL['lag_1_diff'] /df_INFL['lag_1']) * 100
df_INFL['YoY_inflation_perc'] = (df_INFL['lag_12_diff'] / df_INFL['lag_12']) * 100
df_INFL = df_INFL[df_INFL['DATE'] >= '2019-08-01']

           DATE  CPIAUCSL
906  2022-07-01   295.271
907  2022-08-01   295.620


In [46]:
df_INFL[df_INFL['DATE']>='2021-08-01']

,DATE,CPI,lag_1_diff,lag_12_diff,lag_1,lag_12,MoM_inflation_perc,YoY_inflation_perc
895,2021-08-01,273.092,0.908,13.512,272.184,259.580,0.333598,5.205332
896,2021-09-01,274.214,1.122,14.024,273.092,260.190,0.410851,5.389907
897,2021-10-01,276.590,2.376,16.238,274.214,260.352,0.866477,6.236941
898,2021-11-01,278.524,1.934,17.803,276.590,260.721,0.699230,6.828372
899,2021-12-01,280.126,1.602,18.562,278.524,261.564,0.575175,7.096542
900,2022-01-01,281.933,1.807,19.733,280.126,262.200,0.645067,7.525934
901,2022-02-01,284.182,2.249,20.836,281.933,263.346,0.797707,7.912024
902,2022-03-01,287.708,3.526,22.680,284.182,265.028,1.240754,8.557586
903,2022-04-01,288.663,0.955,21.936,287.708,266.727,0.331934,8.224139
904,2022-05-01,291.474,2.811,22.875,288.663,268.599,0.973800,8.516413


In [47]:
# Inflation graph
inflation_line = alt.Chart(df_INFL).mark_line().encode(
    x=alt.X('DATE:T', title=None),
    y=alt.Y('YoY_inflation_perc:Q', title=None, scale=alt.Scale(domain=[0, 10]))
)

# Inflation text
inflation_text = inflation_line.mark_text(align='center',fontSize=11,dy=-10).encode(
        text=alt.Text('YoY_inflation_perc:Q', format='.1f')) 

# Combine
(inflation_line+inflation_text).properties(
    width=900, 
    height=250,
    title={'text':'Inflation Causal Relationship Analysis',
           'fontSize':18}
)

alt.LayerChart(...)

In [48]:
interest_data.head()

,DATE,DFEDTARU
0,2017-10-09,1.25
1,2017-10-10,1.25
2,2017-10-11,1.25
3,2017-10-12,1.25
4,2017-10-13,1.25


In [49]:
# Fed Fund Rate
df_INT = interest_data
df_INT['DATE'] = pd.to_datetime(df_INT['DATE'])
df_INT.reset_index(drop=True, inplace=True)
df_INT = df_INT[df_INT['DATE'] >= '2019-08-01']
df_INT = df_INT[df_INT['DATE'] <= '2022-08-01']

# Dataframe melted
df_INT = df_INT.melt(id_vars=['DATE'],var_name='INDEX')
df_INT.head()

,DATE,INDEX,value
0,2019-08-01,DFEDTARU,2.25
1,2019-08-02,DFEDTARU,2.25
2,2019-08-03,DFEDTARU,2.25
3,2019-08-04,DFEDTARU,2.25
4,2019-08-05,DFEDTARU,2.25


In [59]:
interest_line = alt.Chart(df_INT).mark_line(color='green').encode(
    x=alt.X('DATE:T', title=None),
    y=alt.Y('value:Q', title=None, scale=alt.Scale(domain=[0, 10]))
)

interest_line

alt.Chart(...)

In [60]:
(inflation_line+inflation_text+interest_line).properties(
    width=900, 
    height=250,
    title={'text':'Inflation Causal Relationship Analysis',
           'fontSize':18}
)

alt.LayerChart(...)

In [61]:
# Inflation Inducing Events

# COVID

line_events = {'US COVID Emergency Declaration':'2020-02-03',
                   'Stimulus Round 1':'2020-04-01',
                   'Stimulus Round 2':'2020-12-01',
                   'Stimulus Round 3':'2021-03-01',
                   'US Quantitative Easing 4':'2020-03-01'
                  }

line_events_df = pd.DataFrame(line_events.items(), columns=['Event', 'Date'])

covid_lines = alt.Chart(line_events_df).mark_rule(color='gray', size=2).encode(
    x = 'Date:T')

In [ ]:

covid_area = alt.Chart().mark_rect(color='light gray').encode(
    x = 'Date:T', 
    x2 

In [62]:
(inflation_line+inflation_text+interest_line+covid_lines).properties(
    width=900, 
    height=250,
    title={'text':'Inflation Causal Relationship Analysis',
           'fontSize':18}
)

alt.LayerChart(...)